# working attempt

In [29]:
import json
import sentencepiece as spm

# Load the dataset
with open('data/nl2bash-data.json') as f:
    data = json.load(f)

# # Extract the text and write it to a new file
# with open('data/nl2bash.txt', 'w') as f:
#     for item in data.values():
#         f.write(f"{item['invocation']} \t {item['cmd']}\n")

# Train the tokenizer
spm.SentencePieceTrainer.train('--input=data/nl2bash.txt --model_prefix=m --vocab_size=2500  --pad_id=0')

# Load the trained tokenizer
sp = spm.SentencePieceProcessor()
sp.load('m.model')

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=data/nl2bash.txt --model_prefix=m --vocab_size=2500  --pad_id=0
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/nl2bash.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 2500
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_p

RuntimeError: Internal: src/trainer_interface.cc(758) [insert_id(trainer_spec_.pad_id(), trainer_spec_.pad_piece())] 

In [20]:
import json
import math
from numpy import pad
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
import sentencepiece as spm
import pytorch_lightning as pl
from lightning.pytorch import loggers as pl_loggers

tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs/")
# Load the dataset
with open('data/nl2bash-data.json') as f:
    data = json.load(f)

# # Extract the text and write it to a new file
# with open('text.txt', 'w') as f:
#     for item in data:
#         f.write(item['invocation'] + '\n')
#         f.write(item['cmd'] + '\n')

# # Train the tokenizer
# spm.SentencePieceTrainer.train('--input=text.txt --model_prefix=m --vocab_size=2000')

# Load the trained tokenizer
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# Tokenize the 'invocation' and 'cmd' fields and convert them to integer sequences
inputs = [torch.tensor([sp.bos_id()]+sp.encode_as_ids(item['invocation'])+ [sp.eos_id()], dtype=torch.long) for item in data.values()]
targets = [torch.tensor([sp.bos_id()]+sp.encode_as_ids(item['cmd'])+ [sp.eos_id()], dtype=torch.long) for item in data.values()]

# Pad the sequences to a fixed length
max_length = max(max(len(x) for x in inputs), max(len(x) for x in targets))
inputs = pad_sequence([torch.cat([x, torch.zeros(max_length - len(x), dtype=torch.long)]) for x in inputs], padding_value=-1, batch_first=True)
targets = pad_sequence([torch.cat([x, torch.zeros(max_length - len(x), dtype=torch.long)]) for x in targets], padding_value=-1, batch_first=True)


# Create a PyTorch Dataset and DataLoader
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)
        
    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# Define the Transformer model
class TransformerModel(pl.LightningModule):
    def __init__(self, n_chars, d_model=128, nhead=8, num_layers=6):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(n_chars, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.generator = nn.Linear(d_model, n_chars)
        self.pos_encoder = PositionalEncoding(d_model, 0.1)

    def forward(self, src, tgt=None):
        src_embedding = self.embedding(src) * math.sqrt(self.d_model)
        src_embedding = self.pos_encoder(src_embedding.transpose(0, 1))
        
        if tgt is not None:
            tgt_embedding = self.embedding(tgt) * math.sqrt(self.d_model)
            tgt_embedding = self.pos_encoder(tgt_embedding.transpose(0, 1))
            output = self.transformer(src_embedding, tgt_embedding)
        else:
            output = self.transformer(src_embedding, src_embedding)
        
        output = self.generator(output.transpose(0, 1))
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_input = y[:, :-1]
        y_target = y[:, 1:]
        y_hat = self(x, y_input)
        loss = nn.CrossEntropyLoss()(y_hat.view(-1, y_hat.size(-1)), y_target.contiguous().view(-1))
        self.log('train_loss', loss)

        if batch_idx % 100 == 0:
        # Generate the predicted sequence
            with torch.no_grad():
                y_pred = self.generate(x)

        # Decode the predicted and target sequences
            pred_seq = sp.decode_ids(y_pred[0].tolist())
            target_seq = sp.decode_ids(y[0].tolist())

            print(f"Iteration: {batch_idx}")
            print(f"Predicted sequence: {pred_seq}")
            print(f"Target sequence: {target_seq}")
            print("---")
        
        return loss

    def generate(self, src, max_length=100):
        src_embedding = self.embedding(src) * math.sqrt(self.d_model)
        src_embedding = self.pos_encoder(src_embedding.transpose(0, 1))
        output = src_embedding

        for _ in range(max_length):
            output = self.transformer(src_embedding, output)
            output = self.generator(output[-1:])
            predicted_token = output.argmax(dim=-1)
            output = torch.cat((src_embedding, self.embedding(predicted_token) * math.sqrt(self.d_model)), dim=0)

        return output.transpose(0, 1).argmax(dim=-1)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
        return [optimizer], [scheduler]

# Initialize the model
model = TransformerModel(n_chars=sp.get_piece_size())

# Initialize the trainer
trainer = pl.Trainer(max_epochs=20, log_every_n_steps=10)

# Train the model
trainer.fit(model, dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | embedding   | Embedding          | 256 K 
1 | transformer | Transformer        | 7.5 M 
2 | generator   | Linear             | 258 K 
3 | pos_encoder | PositionalEncoding | 0     
---------------------------------------------------
8.0 M     Trainable params
0         Non-trainable params
8.0 M     Total params
32.114    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Iteration: 0
Predicted sequence: :ir thel folderdepth rmg variable them "*., . . . . . . . . . . . . . . . . . . . . . . . . .dir . . . . . .usr . . . . . . . . . . . . .usr . .usr . . . . . . . . . . . . . . . . . . . .usr . . . . . . .usr . . .usr . . . . . . . . . .usr . . . . . . . .usr . . . . . . .usr . . . . . . . . . . . . . . . . . . . . . . . . . . . .dir . . .usr . . . . .dir . . . . . . . . .dir .dir . . . . . . . . . . last
Target sequence: which lshw ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  

`Trainer.fit` stopped: `max_epochs=20` reached.


In [21]:
prompt = "Your input prompt goes here"
input_ids = sp.encode_as_ids(prompt)
input_tensor = torch.tensor([sp.bos_id()] + input_ids + [sp.eos_id()], dtype=torch.long).unsqueeze(0)

# Generate the output
with torch.no_grad():
    output = model(input_tensor)
    output_ids = output.argmax(dim=-1).squeeze().tolist()

# Decode the generated output tokens
output_text = sp.decode_ids(output_ids)

print("Input prompt:", prompt)
print("Generated response:", output_text)


Input prompt: Your input prompt goes here
Generated response:  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 


In [16]:
output_ids

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
def generate_command(model, sp, text, max_length=15):
    # Tokenize the input text
    input_ids = torch.tensor([sp.encode_as_ids(text)], dtype=torch.long)

    # Move the input to the same device as the model
    input_ids = input_ids.to(next(model.parameters()).device)

    # Initialize the output sequence with the input
    output_ids = input_ids.clone()

    # Generate the output sequence one token at a time
    for i in range(max_length):
        # Get the model's predictions for the next token
        with torch.no_grad():
            predictions = model(output_ids, output_ids)

        # Get the index of the predicted token
        predicted_id = torch.argmax(predictions[0, -1]).unsqueeze(0)

        # Append the predicted token to the output sequence
        output_ids = torch.cat([output_ids, predicted_id.unsqueeze(0)], dim=-1)

        # Stop generating if the end-of-sequence token is predicted
        if predicted_id == sp.eos_id():
            break

    # Decode the output sequence into a string
    command = sp.decode_ids(output_ids[0].tolist())

    return command

In [12]:
text = "list all files in the current directory"
command = generate_command(model, sp, text)
print(command)

list all files in the current directory ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 


t5 model tokenizer

In [4]:
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '<pad>', 'bos_token': '<bos>', 'eos_token': '<eos>'})

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(pl.LightningModule):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, vocab_size)
        self.src_tok_emb = TokenEmbedding(vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self,
                src: torch.Tensor,
                tgt: torch.Tensor,
                src_mask: torch.Tensor,
                tgt_mask: torch.Tensor,
                src_padding_mask: torch.Tensor,
                tgt_padding_mask: torch.Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask)
        return self.generator(outs)

    def encode(self, src: torch.Tensor, src_mask: torch.Tensor):
        return self.transformer.encoder(self.positional_encoding(
            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: torch.Tensor, memory: torch.Tensor, tgt_mask: torch.Tensor):
        return self.transformer.decoder(self.positional_encoding(
            self.tgt_tok_emb(tgt)), memory, tgt_mask)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

    def training_step(self, batch, batch_idx):
        src, tgt = batch
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt[:-1,:])
        logits = self(src, tgt[:-1,:], src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
        tgt_out = tgt[1:, :]
        loss = nn.CrossEntropyLoss()(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        src, tgt = batch
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt[:-1,:])
        logits = self(src, tgt[:-1,:], src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
        tgt_out = tgt[1:, :]
        loss = nn.CrossEntropyLoss()(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        self.log('val_loss', loss)

# Masks
def generate_square_subsequent_mask(sz, device):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = torch.triu(torch.ones((tgt_seq_len, tgt_seq_len), device=src.device)).transpose(0, 1).type(torch.bool)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=src.device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

# Dataloader
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(torch.tensor(tokenizer.encode(src_sample, add_special_tokens=True)))
        tgt_batch.append(torch.tensor(tokenizer.encode(tgt_sample, add_special_tokens=True)))

    src_batch = torch.nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = torch.nn.utils.rnn.pad_sequence(tgt_batch, padding_value=PAD_IDX)


    return src_batch, tgt_batch

# Training
def train(model, train_dataloader, val_dataloader, max_epochs):
    trainer = pl.Trainer(max_epochs=max_epochs)
    trainer.fit(model, train_dataloader, val_dataloader)

# Translation
def translate(model, src_sentence):
    model.eval()
    src = torch.tensor(tokenizer.encode(src_sentence)).unsqueeze(0)
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return tokenizer.decode(tgt_tokens.tolist(), skip_special_tokens=True)

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(model.device)
    src_mask = src_mask.to(model.device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(model.device)
    for i in range(max_len - 1):
        memory = memory.to(model.device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0), model.device).type(torch.bool)).to(model.device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys



In [5]:
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
VOCAB_SIZE = len(tokenizer)
MAX_EPOCHS = 10
GPUS = 1

import json
# Load the JSON data
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, split):
        self.data = data
        self.split = split

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        invocation = item['invocation']
        cmd = item['cmd']
        return invocation, cmd

# Load the JSON data
with open('data/nl2bash-data.json') as f:
    data = json.load(f)
    new = list()
    for _, objectdict in data.items():
        new.append(objectdict)
    
data = new

# Split the data into train, test, and validation subsets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

num_samples = len(data)
num_train = int(num_samples * train_ratio)
num_val = int(num_samples * val_ratio)
num_test = num_samples - num_train - num_val

train_data = data[:num_train]
val_data = data[num_train:num_train+num_val]
test_data = data[num_train+num_val:]

train_dataset = CustomDataset(train_data, split='train')
val_dataset = CustomDataset(val_data, split='val')
test_dataset = CustomDataset(test_data, split='test')

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

# Create your model
model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, VOCAB_SIZE, FFN_HID_DIM)

# Train the model
# pl.Trainer(max_epochs=MAX_EPOCHS).fit(model, train_dataloader, val_dataloader)
train(model, train_dataloader, val_dataloader, MAX_EPOCHS)


# Translate
# src_sentence = "Your input sentence here"
# translated_sentence = translate(model, src_sentence)
# print(f"Translated sentence: {translated_sentence}")


/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                | Type               | Params
-----------------------------------------------------------
0 | transformer         | Transformer        | 12.6 M
1 | generator           | Linear             | 25.8 M
2 | src_tok_emb         | TokenEmbedding     | 25.7 M
3 | tgt_tok_emb         | TokenEmbedding     | 25.7 M
4 | positional_encoding | PositionalEncoding | 0     
-----------------------------------------------------------
89.9 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 42. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 40. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
